# Retrieve JUMP profiles

This is a tutorial on how to access profiles from the [JUMP Cell
Painting datasets](https://github.com/jump-cellpainting/datasets). We
will use polars to fetch the data frames lazily, with the help of `s3fs`
and `pyarrow`. We prefer lazy loading because the data can be too big to
be handled in memory.

In [1]:
import polars as pl

The JUMP Cell Painting project provides several processed datasets for
morphological profiling:

-   **`crispr`**: CRISPR knockout genetic perturbations
-   **`orf`**: Open Reading Frame (ORF) overexpression perturbations
-   **`compound`**: Chemical compound perturbations
-   **`all`**: Combined dataset containing all perturbation types

Each dataset is available in two versions:

-   **Standard**: Fully processed including batch correction
-   **Interpretable**: Same processing but without batch correction
    steps (which involve transformations that lose the original feature
    space)

All datasets are stored as Parquet files on AWS S3 and can be accessed
directly via their URLs. Snakemake workflows for producing these
assembled profiles are available
[here](https://github.com/broadinstitute/jump-profiling-recipe/). The
specific commit used to produce the profiles can be found in the folder
path of each parquet file. For example,
`jump-profiling-recipe_2024_a917fa7` indicates commit `a917fa7` was
used. The index file below contains the exact locations and metadata for
each dataset:

In [2]:
INDEX_FILE = "https://raw.githubusercontent.com/jump-cellpainting/datasets/v0.9.0/manifests/profile_index.csv"

We use the version-controlled CSV above to release the latest corrected
profiles

In [3]:
profile_index = pl.read_csv(INDEX_FILE)
profile_index.head()

We do not need the ‘etag’ (used to check file integrity) column nor the
‘interpretable’ (i.e., before major modifications)

In [4]:
selected_profiles = profile_index.filter(
    pl.col("subset").is_in(("crispr", "orf", "compound"))
).select(pl.exclude("etag"))
filepaths = dict(selected_profiles.iter_rows())
print(filepaths)

{'orf': 'https://cellpainting-gallery.s3.amazonaws.com/cpg0016-jump-assembled/source_all/workspace/profiles/jump-profiling-recipe_2024_a917fa7/ORF/profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony/profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony.parquet', 'crispr': 'https://cellpainting-gallery.s3.amazonaws.com/cpg0016-jump-assembled/source_all/workspace/profiles/jump-profiling-recipe_2024_a917fa7/CRISPR/profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected/profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected.parquet', 'compound': 'https://cellpainting-gallery.s3.amazonaws.com/cpg0016-jump-assembled/source_all/workspace/profiles/jump-profiling-recipe_2024_a917fa7/COMPOUND/profiles_var_mad_int_featselect_harmony/profiles_var_mad_int_featselect_harmony.parquet'}

We will lazy-load the dataframes and print the number of rows and
columns

In [5]:
info = {k: [] for k in ("dataset", "#rows", "#cols", "#Metadata cols", "Size (MB)")}
for name, path in filepaths.items():
    data = pl.scan_parquet(path)
    n_rows = data.select(pl.len()).collect().item()
    schema = data.collect_schema()
    metadata_cols = [col for col in schema.keys() if col.startswith("Metadata")]
    n_cols = schema.len()
    n_meta_cols = len(metadata_cols)
    estimated_size = int(round(4.03 * n_rows * n_cols / 1e6, 0))  # B -> MB
    for k, v in zip(info.keys(), (name, n_rows, n_cols, n_meta_cols, estimated_size)):
        info[k].append(v)

pl.DataFrame(info)

Let us now focus on the `crispr` dataset and use a regex to select the
metadata columns. We will then sample rows and display the overview.
Note that the collect() method enforces loading some data into memory.

In [6]:
data = pl.scan_parquet(filepaths["crispr"])
data.select(pl.col("^Metadata.*$").sample(n=5, seed=1)).collect()

The following line excludes the metadata columns:

In [7]:
data_only = data.select(pl.all().exclude("^Metadata.*$").sample(n=5, seed=1)).collect()
data_only

Finally, we can convert this to `pandas` if we want to perform analyses
with that tool. Keep in mind that this loads the entire dataframe into
memory.

In [8]:
data_only.to_pandas()

5 rows × 259 columns